In [7]:
import numpy as np
import decimal
import matplotlib.pyplot as plt
import sympy as sp

In [8]:
def intializeContext(significantFigs,rounding) :
    #Intialize the context with specified signFigs and rounding/chopping
    decimal.getcontext().prec = significantFigs 
    if( rounding ) :
        decimal.getcontext().rounding = decimal.ROUND_HALF_UP
    else :
        decimal.getcontext().rounding = decimal.ROUND_DOWN 

In [9]:
def parse_exp(expr_str , var_name='x') :
    x = sp.symbols(var_name)
    expr = sp.sympify(expr_str)
    
    def decimal_func(x_decimal) :
        if not isinstance(x_decimal,decimal.Decimal) :
            x_decimal = decimal.Decimal(x_decimal)
            
        result = expr.evalf(decimal.getcontext().prec,subs={x:x_decimal})
        return decimal.Decimal(str(result))
    
    return decimal_func 

In [10]:
def parse_derivative(expr_str , var_name='x' ) :
    x = sp.symbols(var_name)
    expr = sp.sympify(expr_str)
    deriv = sp.diff(expr)
    
    def decimal_derv(x_decimal) :
        if not isinstance(x_decimal,decimal.Decimal) :
            x_decimal = decimal.Decimal(x_decimal)
        result = deriv.evalf(decimal.getcontext().prec,subs = {x:x_decimal})
        return decimal.Decimal(str(result))
    return decimal_derv

In [31]:
def ModifiedNewtonRaphson(x0, func_expr, tol=1e-7, max_iter=100,
                          significantFigs=14, rounding=True):

    intializeContext(significantFigs, rounding=rounding)

    x_root = decimal.Decimal(x0)
    relativeError = decimal.Decimal("Infinity")

    func = parse_exp(func_expr)

    derivative = parse_derivative(func_expr)
    deriv_exp = str(sp.diff(sp.sympify(func_expr)))

    d2_expr = str(sp.diff(sp.sympify(deriv_exp)))
    second_derivative = parse_exp(d2_expr)

    iteration_details = []

    for i in range(max_iter):
        x_root_old = x_root

        fx = func(x_root_old)
        fpx = derivative(x_root_old)
        fppx = second_derivative(x_root_old)

        eps = decimal.Decimal("1e-30")  # tiny number
        denominator = fpx**2 - fx * fppx

        if abs(denominator) < eps:
            # fallback to standard Newton step
            if fpx == 0:  # can't divide
                break   # consider root found
            x_root = x_root_old - fx / fpx
        else:
            x_root = x_root_old - (fx * fpx) / denominator


        x_root = x_root_old - (fx * fpx) / denominator

        if x_root != 0 and i != 0:
            relativeError = abs(x_root - x_root_old)

        expr_eval_str = (
            f"{x_root_old} - ("
            f"({func_expr.replace('x', str(x_root_old))}) * "
            f"({deriv_exp.replace('x', str(x_root_old))}) / "
            f"(({deriv_exp.replace('x', str(x_root_old))})^2 - "
            f"({func_expr.replace('x', str(x_root_old))}) * "
            f"({d2_expr.replace('x', str(x_root_old))})) = {x_root}"
        )

        iteration_details.append({
            "iteration": i + 1,
            "x_root": x_root,
            "Relative Error": relativeError,
            "Evaluation": expr_eval_str
        })

        if relativeError <= tol:
            break

    return x_root, iteration_details


In [32]:
x0 = 0.5
func_expr = "(x - 1)**3"

root, details = ModifiedNewtonRaphson(
    x0=x0,
    func_expr=func_expr,
    tol=1e-7,
    max_iter=20,
    significantFigs=14,
    rounding=True
)

print("Computed Root:", root)
for row in details:
    print(row)

Computed Root: 1.0000000000000
{'iteration': 1, 'x_root': Decimal('1.0'), 'Relative Error': Decimal('Infinity'), 'Evaluation': '0.5 - (((0.5 - 1)**3) * (3*(0.5 - 1)**2) / ((3*(0.5 - 1)**2)^2 - ((0.5 - 1)**3) * (6*0.5 - 6)) = 1.0'}
{'iteration': 2, 'x_root': Decimal('1.0000000000000'), 'Relative Error': Decimal('0E-13'), 'Evaluation': '1.0 - (((1.0 - 1)**3) * (3*(1.0 - 1)**2) / ((3*(1.0 - 1)**2)^2 - ((1.0 - 1)**3) * (6*1.0 - 6)) = 1.0000000000000'}


In [36]:
test_cases = [
    ("(x-1)**3", 0.5, 1),
    ("x**2 - 4", 3, 2),
    ("x**3 - 3*x**2 + 3*x - 1", 0, 1),
    ("x**3 + 2*x - 5", 1, 1.094551),
    ("x**4", 0.1, 0),
    ("sin(x)", 3, 3.14159265),
    ("exp(x) - 2", 0.5, 0.69314718),
]

for expr, x0, exact in test_cases:

    try:
        root, details = ModifiedNewtonRaphson(x0, expr, tol=1e-14, max_iter= 1000)
        print(f"f(x) = {expr}, x0 = {x0}")
        print(f"  Computed root: {root}")
        print(f"  Expected root: {exact}")
        print(f"  Relative error: {abs(root - decimal.Decimal(str(exact)))}\n")
        for row in details:
            print(row)
         
    except:
        print("Div by Zero")


f(x) = (x-1)**3, x0 = 0.5
  Computed root: 1.0000000000000
  Expected root: 1
  Relative error: 0E-13

{'iteration': 1, 'x_root': Decimal('1.0'), 'Relative Error': Decimal('Infinity'), 'Evaluation': '0.5 - (((0.5-1)**3) * (3*(0.5 - 1)**2) / ((3*(0.5 - 1)**2)^2 - ((0.5-1)**3) * (6*0.5 - 6)) = 1.0'}
{'iteration': 2, 'x_root': Decimal('1.0000000000000'), 'Relative Error': Decimal('0E-13'), 'Evaluation': '1.0 - (((1.0-1)**3) * (3*(1.0 - 1)**2) / ((3*(1.0 - 1)**2)^2 - ((1.0-1)**3) * (6*1.0 - 6)) = 1.0000000000000'}
f(x) = x**2 - 4, x0 = 3
  Computed root: 2.0000000000000
  Expected root: 2
  Relative error: 0E-13

{'iteration': 1, 'x_root': Decimal('1.8461538461538'), 'Relative Error': Decimal('Infinity'), 'Evaluation': '3 - ((3**2 - 4) * (2*3) / ((2*3)^2 - (3**2 - 4) * (2)) = 1.8461538461538'}
{'iteration': 2, 'x_root': Decimal('1.9936102236422'), 'Relative Error': Decimal('0.1474563774884'), 'Evaluation': '1.8461538461538 - ((1.8461538461538**2 - 4) * (2*1.8461538461538) / ((2*1.846153846